In [1]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--image_size', type=int, default=256, help='Image size of training images')
parser.add_argument('--style_image_size', type=int, default=256, help='Image size of the style (if equal to 0, will not resize). Resizing leads to very different results, I recommended using 256, 512 or 0 and see which one you prefer.')
parser.add_argument('--batch_size', type=int, default=8, help='Batch size, paper use 4.')
parser.add_argument('--n_colors', type=int, default=3)
parser.add_argument('--H_size', type=int, default=16, help='Number of filters in the image transformator. Paper use 32 but official implementation recommend only using 16 for faster speed while retaining quality.')
parser.add_argument('--Residual_blocks', type=int, default=5, help='Number of residual blocks')
parser.add_argument('--SELU', type=bool, default=False, help='Using scaled exponential linear units (SELU) which are self-normalizing instead of ReLU with BatchNorm. Do not use.')
parser.add_argument('--norm_type', default='instance', help='If "instance" uses instance normalization; if "batch" uses batch normalization.')
parser.add_argument('--padding', default='reflect', help='If "reflect" uses reflection padding; if "zero" uses zero padding.')
parser.add_argument('--lr', type=float, default=.001, help='Learning rate')
parser.add_argument('--n_epoch', type=int, default=2, help='Number of epochs. 2 is generally enough, but more can be better, just Ctrl-C to stop it when you feel like it is good enough.')
parser.add_argument('--beta1', type=float, default=0.9, help='Adam betas[0]')
parser.add_argument('--beta2', type=float, default=0.999, help='Adam betas[1]')
parser.add_argument("--content_weight", type=float, default=1, help="Weight of content loss")
parser.add_argument("--style_weight", type=float, default=5, help="Weight of style loss. Make bigger or smaller depending on observed results vs desired results.")
parser.add_argument("--total_variation_weight", type=float, default=1e-6, help="Weight of total variation loss (Should be between 1e-4 and 1e-6)")
parser.add_argument("--feature", type=int, default=1, help="Contant loss feature used: 0=relu1_2, 1=relu2_2, 2=relu3_3, 3=relu4_3. Paper use relu2_2, official implementation use relu3_3.")
parser.add_argument("--NN_conv", type=bool, default=False, help="This is highly recommended. This approach minimize checkerboard artifacts during training. Uses nearest-neighbor resized convolutions instead of strided convolutions (https://distill.pub/2016/deconv-checkerboard/ and github.com/abhiskk/fast-neural-style).")
parser.add_argument("--ANTIALIAS", type=bool, default=False, help="Use antialiasing instead of bilinear to resize images. Sightly slower but sightly better quality.")
parser.add_argument('--seed', type=int)
parser.add_argument('--input_folder', default='/mnt/sdb2/Datasets/COCO', help='input folder (Coco dataset for training, whichever dataset after to apply style)')
parser.add_argument('--style_picture', default='/mnt/sdb2/styles/candy.jpg', help='Style picture')
parser.add_argument('--VGG16_folder', default='/mnt/sdb2/VGG16', help='folder for the VGG16 (Will be downloaded automatically into this folder)')
parser.add_argument('--output_folder', default='/mnt/sdb2/Output/FNT', help='output folder')
parser.add_argument('--model_load', default='', help='Full path to transformation model to load (ex: /home/output_folder/run-5/models/G_epoch_11.pth)')
parser.add_argument('--trained_model', type=bool, default=False, help='If True, the model has been trained and we only want it to generate the pictures (not resized).')
parser.add_argument('--cuda', type=bool, default=True, help='enables cuda')
parser.add_argument('--n_gpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--n_workers', type=int, default=2, help='Number of subprocess to use to load the data. Use at least 2 or the number of cpu cores - 1.')


args, unknown = parser.parse_known_args()

In [ ]:
wb_run = wandb.init(project="cat-generation-FastNeuralTransfer",config={"architecture": "FastNeuralTransfer"})
wandb.config.update(args) # adds all of the arguments as config variables
param = wandb.config

print(param)

In [2]:
## Imports

import time
start = time.time()
import os
run=0
base_dir = f"{param.output_folder}/run-{run}/"
while os.path.exists(base_dir):
    run+=1
    base_dir = f"{param.output_folder}/run-{run}"
os.makedirs(base_dir)
os.makedirs(f"{base_dir}/models")

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transf
import torchvision.models as models
import torchvision.utils as vutils
from torch.utils.data import Dataset
from natsort import natsorted
# from PIL import Image
import PIL
import imageio as iio

from torch.utils.serialization import load_lua

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if param.cuda:
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True
    
from IPython.display import Image
to_img = transf.ToPILImage()

import math
import itertools
import random
import numpy as np

import random
param.seed = param.seed or random.randint(1,10000)
print(f"Random Seed: {param.seed}")
wandb.log({"Random": param.seed})
random.seed(param.seed)
torch.manual_seed(param.seed)

if param.cuda:
    torch.cuda.manual_seed_all(param.seed)
    
## Transforming images
trans = []
if not param.trained_model:
    if param.ANTIALIAS:
        trans.append(transf.Scale(param.image_size,PIL.Image.ANTIALIAS))
    else:
        trans.append(transf.Scale(param.image_size,PIL.Image.BILINEAR))
    trans.append(transfCenterCrop(param.image_size))
trans.append(transf.ToTensor())
trans.append(transf.ToTensor())
trans.append(transf.Lambda(lambda x: x.mul(255)))
trans = transf.Compose(trans)

trans_style = []
if param.style_image_size>0:
    if param.ANTIALIAS:
        trans_style.append(transf.Scale(param.style_image_size,PIL.Image.ANTIALIAS))
    else:
        trans_style.append(transf.Scale(param.style_image_size,PIL.Image.BILINEAR))
    trans_style.append(transf.CenterCrop(param.style_image_size))
trans_style.append(transf.ToTensor())
trans_style.append(transf.Lambda(lambda x: x.mul(255)))
trans_style = transf.Compose(trans_style)
if param.trained_model:
    param.batch_size=1

class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = PIL.Image.open(img_loc).convert("RGB")
        # image = iio.imread(img_loc)
        tensor_image = self.transform(image)
        return tensor_image
    
my_dataset = CustomDataSet(param.input_folder, transform=trans)
dataset = torch.utils.data.DataLoader(my_dataset , batch_size=param.batch_size, shuffle=True, 
                               num_workers=param.n_workers)

style_picture = PIL.Image.open(param.style_picture)
style_picture = trans_style(style_picture)

## Models
# Reflection padding is an alternative to 0 padding (like looking at water reflection)
# Official implementation of the paper use only reflection paddding in the downsample block but I use it everywhere, I'm not sure if it makes a difference.

# Set batch norm or instance norm
import functools
if param.norm_type = 'batch':
    Norm2D = functools.partial(nn.BatchNormed)
elif param.norm_type = 'instance':
    Norm2D = functools.partial(nn.InstanceNorm2d)
    
# Padding
if param.padding == "reflect":
    pad=0
if param.padding == "zero":
    pad = 1
    
# Residual Block of Generator
class Residual_block(torch.nn.Module):
    def __init__(self,h_size):
        super(Residual_block,self).__init__()
        # Two conv layer with same output size
        model=[]
        if param.padding =="reflect":
            model+=[nn.ReflectionPad2d(padding=1)]
        model+=[nn.Conv2d(h_size,h_size,kernel_size=3,stride=1,padding=pad)]
        if param.SELU:
            model+=[torch.nn.SELU()]
        else:
            model+=[Norm2D(h_size),nn.ReLU(True)]
        if param.padding =="reflect":
            model+=[nn.ReflectionPad2d(padding=1)]
        model += [nn.Conv2d(h_size,h_size,kernel_size=3,stride=1,padding=pad)]
        if not param.SELU:
            model+=[Norm2D(h_size)]
        self.model=nn.Sequential(*model)
        
    def forward(self,input):
        # Return itself + the result of the two convolutions
        output = self.model(input)+input
        return output

SyntaxError: invalid syntax (1043885275.py, line 113)